## Markov Decision Process

- 给定一个马尔可夫过程, 可以从某个状态出发, 根据它的状态转移矩阵生成一个状态序列(episode), 这个步骤也被叫做采样(sampling)

### 马尔可夫奖励过程
马尔可夫奖励过程（Markov Reward Process, MRP）由元组 $\langle S, P, R, \gamma \rangle$ 组成：
- $S$ 是有限状态集
- $P$ 是状态转移概率矩阵，$P(s'|s) = \mathbb{P}[S_{t+1}=s'|S_t=s]$
- $R$ 是奖励函数，$R(s) = \mathbb{E}[R_{t+1}|S_t=s]$
- $\gamma$ 是折扣因子，$\gamma \in [0,1]$

**回报（Return）**定义：
$$ G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^\infty \gamma^k R_{t+k+1} $$

**状态价值函数（State Value Function）**：
$$ V(s) = \mathbb{E}[G_t|S_t=s] $$

**贝尔曼方程**：
$$ V(s) = R(s) + \gamma \sum_{s' \in S} P(s'|s)V(s') $$

In [4]:
import numpy as np
np.random.seed(0)
# 定义状态转移概率矩阵P
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0]  # 定义奖励函数
gamma = 0.5  # 定义折扣因子


# 给定一条序列,计算从某个索引（起始状态）开始到序列最后（终止状态）得到的回报
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i] - 1]
    return G


# 一个状态序列,s1-s2-s3-s6
chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print("根据本序列计算得到回报为：%s。" % G)

根据本序列计算得到回报为：-2.5。


In [5]:
# V = (I - γP)^(-1) * R
def compute(P, rewards, gamma, states_num):
    ''' 利用贝尔曼方程的矩阵形式计算解析解,states_num是MRP的状态数 '''
    rewards = np.array(rewards).reshape((-1, 1))  #将rewards写成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P),
                   rewards)
    return value


V = compute(P, rewards, gamma, 6)
print("MRP中每个状态价值分别为\n", V)

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


## 马尔可夫决策过程
- 智能体的策略（Policy）通常用字母$\pi$表示
- 状态价值函数: $V^\pi(s) = \mathbb{E}[G_t|S_t=s, A_t \sim \pi]$
- 动作价值函数: $Q^\pi(s,a) = \mathbb{E}[G_t|S_t=s, A_t=a, A_{t+1} \sim \pi]$
- 状态价值函数和动作价值函数之间的关系: $V^\pi(s) = \sum_a \pi(a|s)Q^\pi(s,a)$

## 贝尔曼期望方程
- 状态价值函数: $V^\pi(s) = \sum_a \pi(a|s) \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a)V^\pi(s') \right]$
- 动作价值函数: $Q^\pi(s,a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a) \sum_a \pi(a|s')Q^\pi(s',a)$

In [7]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}


# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2